In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<AxesSubplot:xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<AxesSubplot:xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

domain         \
                                                    count unique   
domain                       posted_by                             
americanthinker.com          MtnClimber                66      1   
                             SeekAndFind               47      1   
                             SoConPubbie                1      1   
                             ebb tide                   1      1   
apnews.com                   Morgana                    1      1   
                             nickcarraway               1      1   
breitbart.com                SoConPubbie               39      1   
                             Morgana                    8      1   
                             CFW                        6      1   
                             ChicagoConservative27    165      1   
                             Red Badger                 1      1   
dailymail.co.uk              Morgana                   62      1   
                             nickcarraway               2      1   
                             CFW                        2      1   
                             Red Badger                 6      1   
                             Libloather                65      1   
foxnews.com                  Morgana                    2      1   
                             nickcarraway              13      1   
                             CFW                        8      1   
                             ChicagoConservative27     16      1   
                             Libloather                19      1   
hotair.com                   MtnClimber                 1      1   
                             SeekAndFind               39      1   
justthenews.com              SoConPubbie                1      1   
                             CFW                       17      1   
                             Red Badger                 3      1   
                             Libloather                 1      1   
lifesitenews.com             ebb tide                  24      1   
                             Morgana                    7      1   
msn.com                      SeekAndFind                2      1   
                             nickcarraway               3      1   
                             CFW                        1      1   
                             Red Badger                 1      1   
nypost.com                   SeekAndFind                2      1   
                             SoConPubbie                2      1   
                             Morgana                    1      1   
                             nickcarraway              59      1   
                             CFW                        5      1   
                             ChicagoConservative27    111      1   
                             Red Badger                 4      1   
                             Libloather                 1      1   
pjmedia.com                  SeekAndFind               29      1   
                             Red Badger                 1      1   
redstate.com                 SeekAndFind               50      1   
                             CFW                        2      1   
                             Red Badger                 1      1   
rumble.com                   Morgana                    2      1   
                             CFW                        2      1   
theconservativetreehouse.com SoConPubbie               29      1   
theepochtimes.com            SeekAndFind               19      1   
                             CFW                       10      1   
                             Red Badger                 1      1   
thefederalist.com            SeekAndFind               19      1   
                             CFW                        2      1   
                             Red Badger                 1      1   
thegatewaypundit.com         SoConPubbie                1      1   
                             Red Badger

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

,timestamp,replies
title,,
Kerch Bridge Collapse Following Explosions,2023-07-17 06:27:59,212.0
𝙊𝙈𝙂!!! 𝙇𝘼𝙕𝘼𝙈𝘼𝙏𝘼𝙕 𝘿𝙄𝙀𝘿 𝙏𝙊𝙉𝙄𝙂𝙃𝙏!!!!,2023-07-23 09:11:26,193.0
"DeSantis says Trump not only didn’t fulfill major promise, it got worse",2023-07-17 00:48:05,179.0
Obamas’ personal chef drowns near their Martha’s Vineyard home,2023-07-25 00:17:10,167.0
